In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import time
import pickle
from imp import reload
from os.path import join

import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from data_loader import DataLoader
import medim
reload(medim);

In [14]:
def pred_reshape(y):
    x = y.permute(0, 2, 3, 4, 1)
    return x.contiguous().view(-1, x.size()[-1])

def to_var(x):
    return Variable(torch.from_numpy(x))

def to_numpy(x):
    return x.cpu().data.numpy()

In [ ]:
torch

In [3]:
def dice_score(y_pred, target):
    """Dice score for binary segmentation on 3d scan"""
    return 2 * np.sum(y_pred * target) / (np.sum(y_pred) + np.sum(target))

In [26]:
def dice_loss(y_pred, target):
    coeff = to_var(np.array([1, 2, 3], dtype=np.float32))
    
    y_pred = y_pred.view(*y_pred.size()[:2], -1)
    target = target.view(*target.size()[:2], -1)
    
    dice_scores = 2 * sum(y_pred * target, 2) / \
                  (sum(y_pred, 2) + sum(target, 2))
        
    dice_scores = dice_scores.mean(0)
    dice_scores = dice_scores.view(dice_scores.size()[1])

    return torch.sum(coeff * dice_scores)

In [12]:
tmp = np.float32(np.random.randint(2, size=(10, 3, 8, 8, 8)))
tmp.shape

(10, 3, 8, 8, 8)

In [28]:
def dice_loss(y_pred, target):
    coeff = to_var(np.array([1, 2, 3], dtype=np.float32))
    
    y_pred = y_pred.view(*y_pred.size()[:2], -1)
    target = target.view(*target.size()[:2], -1)
    
    dice_scores = 2 * (y_pred * target).sum(2) / \
                  (y_pred.sum(2) + target.sum(2))
        
    dice_scores = dice_scores.mean(0)
    dice_scores = dice_scores.view(-1)

    return torch.sum(dice_scores * coeff)

In [29]:
dice_loss(to_var(tmp), to_var(tmp))

Variable containing:
 6
[torch.FloatTensor of size 1]

In [108]:
example = torch.rand(1, 3, 2, 2, 2)

In [76]:
example.size()

torch.Size([1, 3, 2, 2, 2])

In [77]:
example = example.view(*example.size()[:2], -1)

In [78]:
example.size()

torch.Size([1, 3, 8])

In [85]:
example = example.sum(2)

In [86]:
example = sum(example, 2)

In [87]:
example.size()

torch.Size([1, 3])

In [69]:
example = example.mean(0)

In [88]:
example = example.view(*example.size()[:2])

In [72]:
example[0]


 3.3946
 4.8465
 3.9411
[torch.FloatTensor of size 3]

In [89]:
example.size()

torch.Size([1, 3])

In [90]:
example * example


 13.2408  16.8009  29.3152
[torch.FloatTensor of size 1x3]

In [104]:
dice_loss(example, example)

28.07962679862976

In [25]:
a = torch.cuda.device(0)

In [30]:
!ls /mount/export

animals  Brats2017  dsb2017  ndsb2017
